In [1]:
import pandas as pd
import geopandas as gpd
import folium

# === Caminhos ===
caminho_csv = 'https://raw.githubusercontent.com/IgorNzz/Mapa-Sisagua/main/dados_exportados.csv'
caminho_geojson = 'https://raw.githubusercontent.com/IgorNzz/Mapa-Sisagua/main/Delimita%C3%A7%C3%A3o_dos_Bairros_-_Dec._32.791_2020.geojson'
saida_html = 'index.html'

# === Leitura dos dados ===
df = pd.read_csv(caminho_csv)
gdf = gpd.read_file(caminho_geojson)

# === Normalização ===
df['area'] = df['area'].astype(str).str.upper().str.strip()
df['parametro'] = df['parametro'].astype(str).str.strip()
df['resultado'] = df['resultado'].astype(str).str.strip()
df['tipo_da_forma_de_abastecimento'] = df['tipo_da_forma_de_abastecimento'].astype(str).str.strip()
gdf['nome_bairr'] = gdf['nome_bairr'].astype(str).str.upper().str.strip()

# === Filtros e limpeza ===
parametros_alvo = [
    'Escherichia coli',
    'Turbidez (uT)',
    'Cloro residual livre (mg/L)',
    'Coliformes totais',
    'Fluoreto (mg/L)'
]
df = df[df['parametro'].isin(parametros_alvo)]
df = df.dropna(subset=['latitude', 'longitude', 'resultado'])

# === Função para avaliar se está dentro ou fora do padrão ===
def avaliar_amostra(row):
    param = row['parametro']
    val = str(row['resultado']).replace(',', '.').strip().upper()
    
    try:
        num = float(val) if val not in ['AUSENTE', 'PRESENTE'] else None
    except:
        num = None

    if param == 'Cloro residual livre (mg/L)':
        return 'Dentro do padrão' if num is not None and 0.2 <= num <= 5.0 else 'Fora do padrão'
    elif param == 'Turbidez (uT)':
        return 'Dentro do padrão' if num is not None and num <= 5.0 else 'Fora do padrão'
    elif param == 'Escherichia coli':
        return 'Dentro do padrão' if val == 'AUSENTE' else 'Fora do padrão'
    elif param == 'Coliformes totais':
        return 'Dentro do padrão' if val == 'AUSENTE' else 'Fora do padrão'
    elif param == 'Fluoreto (mg/L)':
        return 'Dentro do padrão' if num is not None and 0.6 <= num <= 1.5 else 'Fora do padrão'
    else:
        return 'Desconhecido'

# === Aplica avaliação ===
df['situacao'] = df.apply(avaliar_amostra, axis=1)

# === Criação do mapa ===
mapa = folium.Map(location=[-12.97, -38.50], zoom_start=11, tiles='CartoDB positron')

# === Adiciona os bairros como fundo cinza claro ===
folium.GeoJson(
    gdf,
    name="Bairros",
    style_function=lambda x: {
        'fillColor': 'lightgray',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.2
    },
    tooltip=folium.GeoJsonTooltip(fields=['nome_bairr'], aliases=['Bairro:'])
).add_to(mapa)

# === Adiciona marcadores por amostra ===
for _, row in df.iterrows():
    try:
        lat = float(row['latitude'])
        lon = float(row['longitude'])
        cor = 'green' if row['situacao'] == 'Dentro do padrão' else 'red'

        popup_text = f"""
        <strong>Bairro:</strong> {row['area']}<br>
        <strong>Data da coleta:</strong> {row['data_da_coleta']}<br>
        <strong>Forma de abastecimento:</strong> {row['tipo_da_forma_de_abastecimento']}<br>
        <strong>Parâmetro:</strong> {row['parametro']}<br>
        <strong>Resultado:</strong> {row['resultado']}<br>
        <strong>Situação:</strong> {row['situacao']}
        """
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            color=cor,
            fill=True,
            fill_opacity=0.8,
            popup=folium.Popup(popup_text, max_width=300)
        ).add_to(mapa)
    except:
        continue

# === Camada de controle ===
folium.LayerControl().add_to(mapa)

# === Adiciona a legenda diretamente no mapa ===
legenda_html = """
<div style="
    position: fixed;
    bottom: 30px;
    left: 30px;
    z-index: 9999;
    background-color: white;
    border: 2px solid #444;
    border-radius: 10px;
    padding: 15px;
    width: 320px;
    font-size: 13px;
">
    <h4 style="margin-top: 0;">Legenda - Padrões da Qualidade da Água</h4>
    <div><span style="background-color: green; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></span> Dentro do padrão</div>
    <div><span style="background-color: red; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></span> Fora do padrão</div>
    <hr>
    <strong>Critérios por parâmetro:</strong>
    <ul style="padding-left: 16px; margin-bottom: 0;">
        <li><strong>Cloro Residual Livre:</strong> 0,2 - 5,0 mg/L</li>
        <li><strong>Turbidez:</strong> até 5 UNT</li>
        <li><strong>Escherichia coli:</strong> Ausência em 100 ml</li>
        <li><strong>Coliformes totais:</strong> Ausência em 100 ml</li>
        <li><strong>Fluoreto:</strong> 0,6 - 1,5 mg/L</li>
    </ul>
</div>
"""
mapa.get_root().html.add_child(folium.Element(legenda_html))

# === Salva o mapa ===
mapa.save(saida_html)
print(f"✅ Mapa gerado com sucesso: {saida_html}")


✅ Mapa gerado com sucesso: index.html
